# Programming for Data Analytics 2022, Assignment 2

Undertake an analysis/review of the dataset and present an overview and background.

- Provide a literature review on classifiers which have been applied to the dataset and
compare their performance
- Present a statistical analysis of the dataset
- Using a range of machine learning algorithms, train a set of classifiers on the dataset (using
SKLearn etc.) and present classification performance results. Detail your rationale for the
parameter selections you made while training the classifiers.
- Compare, contrast and critique your results with reference to the literature
- Discuss and investigate how the dataset could be extended – using data synthesis of new
tumour datapoin